Implementation of the Welsh-Powell Algorithm

Sources:

Watson, Gordon. “Comparing the Efficiency of Welsh-Powell Algorithm and Basic Greedy Algorithm for Graph Coloring - Graph Theory.” CopyProgramming, 15 May 2023, copyprogramming.com/howto/why-is-welsh-powell-algorithm-better-than-the-basic-greedy-algorithm-for-graph-coloring.

“Welsh Powell Graph Colouring Algorithm.” GeeksforGeeks, GeeksforGeeks, 29 Oct. 2019, www.geeksforgeeks.org/welsh-powell-graph-colouring-algorithm/.

“Vertex Coloring: The Welsh Powell Algorithm.” Kleemans.Ch, www.kleemans.ch/static/fourcolors/welsh-powell.pdf. Accessed 16 Oct. 2023.

“Graph Coloring Using Greedy Algorithm.” GeeksforGeeks, GeeksforGeeks, 10 Oct. 2023, www.geeksforgeeks.org/graph-coloring-set-2-greedy-algorithm/?ref=ml_lbp\%27.

In [ ]:
sdafjawirjgiojaejiaergoijaeg